In [1]:
import sys
import asyncio
import time

import numpy as np

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [2]:
# Temporarily needed to run at summit
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
#get classes and start them
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

Read historical data in 0.01 sec
Read historical data in 0.35 sec
Read historical data in 0.81 sec
Read historical data in 1.16 sec
Read historical data in 2.09 sec
Read historical data in 3.13 sec
Read historical data in 4.66 sec
Read historical data in 6.04 sec
Read historical data in 6.88 sec
Read historical data in 8.47 sec
Read historical data in 9.51 sec
RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 36 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages


[[None, None, None, None, None, None, None], [None, None, None, None]]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 20 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 41 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 19 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 23 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 47 messages


In [5]:
# enable components
#await attcs.enable()
#await latiss.enable()

In [6]:
# take event checking out the slew commands to test telescope only
attcs.check.atdome = False
attcs.check.atdometrajectory = False

In [7]:
# Perform Rotator verification (LTS-337-003)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 25 messages


In [8]:
await attcs.stop_tracking()

Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.


In [9]:
start_az=165.5
start_el=67.9
start_rot_pa=-170
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)

Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -000.029 | delta Az = -000.054 delta N1 = -000.000 delta N2 = -152.528 [Dome] delta Az = -003.040
[Telescope] delta Alt = +000.000 | delta Az = +000.001 delta N1 = -000.000 delta N2 = -146.893 [Dome] delta Az = -003.040
[Telescope] delta Alt = -000.000 | delta Az = +000.001 delta N1 = -000.000 delta N2 = -140.891 [Dome] delta Az = -003.040
[Telescope] delta Alt = -000.000 | delta Az = +000.000 delta N1 = -000.000 delta N2 = -134.889 [Dome] delta Az = -003.040
[Telescope] delta Alt = -000.000 | delta Az = +000.000 delta N1 = +000.000 delta N2 = -128.895 [Dome] delta Az = -003.040
[Telescope] delta Alt = -000.000 | delta Az = -000.000 d

In [10]:
start_az=165.5
start_el=67.9
start_rot_pa=170

start_time =time.time()
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)
end_time = time.time()
print(f'Time to perform {170*2} degree rotation of N2 is {end_time-start_time} seconds')

Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -000.002 | delta Az = +000.000 delta N1 = +000.000 delta N2 = -020.230 [Dome] delta Az = -003.040
[Telescope] delta Alt = +000.000 | delta Az = +000.000 delta N1 = +000.000 delta N2 = -025.440 [Dome] delta Az = -003.040
[Telescope] delta Alt = +000.000 | delta Az = +000.000 delta N1 = +000.000 delta N2 = -031.441 [Dome] delta Az = -003.040
[Telescope] delta Alt = +000.000 | delta Az = +000.000 delta N1 = +000.000 delta N2 = -037.438 [Dome] delta Az = -003.040
[Telescope] delta Alt = +000.000 | delta Az = +000.000 delta N1 = -000.000 delta N2 = -043.442 [Dome] delta Az = -003.04

In [ ]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

In [ ]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=50
start_el=20
start_rot_pa=0
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)

In [ ]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [ ]:
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
# get RA/DEC of current position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
#print(curr_time_atptg)
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))


In [ ]:
# Why does doing no slew at all take ~32.7 sec?

#send to starting position
await attcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot_pa=0.0, slew_timeout=240., stop_before_slew=True)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image
await latiss.take_engtest(exptime=1)

print('Starting to Slew to target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec), rot_pa=d_rot, slew_timeout=240., stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

In [ ]:
await attcs.stop_tracking()

In [ ]:
# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
await attcs.slew_object('WD 1327-083')

In [ ]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
# stopped tracking elsewhere, then reslewed
await attcs.slew_object('HD 59468')

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

In [ ]:
# again
await attcs.offset_azel(az=-20*60*60, el=0)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-5*60*60, el=0)
await attcs.offset_azel(az=-2*60*60, el=0)
await attcs.offset_azel(az=0, el=0)

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')
    

In [11]:
await latiss.take_bias(nbias=1)

Generating group_id
BIAS 0001 - 0001


array([2020031600076])

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 20 messages


In [ ]:
#await attcs.startup()

In [12]:
await latiss.enable()

Gathering settings.
No settings for atcamera.
Using  from settingVersions event.
No settings for atspectrograph.
Using current from settingVersions event.
No settings for atheaderservice.
Couldn't get settingVersions event. Using empty settings.
No settings for atarchiver.
Using normal from settingVersions event.
Settings versions: {'atcamera': '', 'atspectrograph': 'current', 'atheaderservice': '', 'atarchiver': 'normal'}
Enabling all components
Enabling  atcamera
Enabling  atspectrograph
Enabling  atheaderservice
Enabling  atarchiver
[atcamera]::[<State.ENABLED: 2>]
[atspectrograph]::[<State.ENABLED: 2>]
[atheaderservice]::[<State.ENABLED: 2>]
[atarchiver]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components enabled.
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messag